In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [14]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

from sklearn.preprocessing import StandardScaler,OneHotEncoder

## Import Provisional CSV and Perform Basic Data Cleaning

In [15]:
# Load the data
file_name = "Resources/Diamonds_Prices2022.csv"
df = pd.read_csv(file_name, index_col = "Unnamed: 0")

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')


(53943, 10)


,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [ ]:
# EDA showed no missing values

# # Drop the null columns where all values are null
# df = df.dropna(axis='columns', how='all')

# # Drop the null rows
# df = df.dropna()

#### Address z == 0
Missing values in x, y, and z were assigned as 0's

In [ ]:
# all x and y 0's also have z 0's

# find (x,y,z) 0's
df[df.z == 0]

In [ ]:
# z can not be recovered without x, 
# as round cut diamonds can be quite oval

df.drop(df[df.x == 0].index, inplace=True)

df[df.z == 0]

In [ ]:
# z can be recovered from x, y, and depth:
# z = depth / 100 * mean(x, y)

index_values = df[df.z == 0].index.values
print(index_values)

df.z = np.where(df.z == 0, df.depth / 100 * ((df.x + df.y) / 2), df.z)

# updated z values that were previously 0's
df.loc[index_values]

#### Possible feature selection
- Combine colors I + J, as they contain relatively few values..
- Combine clarities IF + VVS1 and I1 + SI2, as IF and I1 have relatively few values, and IF has a similar clarity to VVS21, and I1 has a similar clarity to SI2 

In [16]:
# encode feature columns that are strings/objects
# use get_dummies

type_objs = ['cut', 'color', 
             'clarity'
            ]

df = pd.get_dummies(df, columns=type_objs)

print(df.shape)
df.head()

(53943, 27)


,carat,depth,table,price,x,y,z,cut_Fair,cut_Good,cut_Ideal,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
1,0.23,61.5,55.0,326,3.95,3.98,2.43,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,0.21,59.8,61.0,326,3.89,3.84,2.31,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0.23,56.9,65.0,327,4.05,4.07,2.31,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,0.29,62.4,58.0,334,4.20,4.23,2.63,0,0,0,...,1,0,0,0,0,0,0,1,0,0
5,0.31,63.3,58.0,335,4.34,4.35,2.75,0,1,0,...,0,1,0,0,0,1,0,0,0,0


In [17]:
df.columns

Index(['carat', 'depth', 'table', 'price', 'x', 'y', 'z', 'cut_Fair',
       'cut_Good', 'cut_Ideal', 'cut_Premium', 'cut_Very Good', 'color_D',
       'color_E', 'color_F', 'color_G', 'color_H', 'color_I', 'color_J',
       'clarity_I1', 'clarity_IF', 'clarity_SI1', 'clarity_SI2', 'clarity_VS1',
       'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2'],
      dtype='object')

# Split the Data into Training and Testing

In [18]:
# Create our features
X = df.copy()
X = X.drop('price', axis=1)

# Create our target
y = df["price"].values

In [19]:
X.describe()

,carat,depth,table,x,y,z,cut_Fair,cut_Good,cut_Ideal,cut_Premium,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
count,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000,...,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000
mean,0.797935,61.749322,57.457251,5.731158,5.734526,3.538730,0.029846,0.090948,0.399514,0.255696,...,0.100514,0.052055,0.013737,0.033183,0.242237,0.170439,0.151475,0.227258,0.067757,0.093914
std,0.473999,1.432626,2.234549,1.121730,1.142103,0.705679,0.170165,0.287538,0.489803,0.436256,...,0.300686,0.222140,0.116397,0.179116,0.428441,0.376022,0.358514,0.419065,0.251330,0.291712
min,0.200000,43.000000,43.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,4.710000,4.720000,2.910000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.700000,61.800000,57.000000,5.700000,5.710000,3.530000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.040000,62.500000,59.000000,6.540000,6.540000,4.040000,0.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.010000,79.000000,95.000000,10.740000,58.900000,31.800000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
# Check the balance of our target values
# Counter(y)

In [21]:
# Splitting into Train and Test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=1)

In [22]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)